<div Style="text-align: center;line-height: 30px;font-size:24px;font-weight: bold"> Etiquetador de imágenes</div>
***

In [44]:
from __future__ import print_function
#Importss
%matplotlib inline
import numpy as np

In [45]:
import ipywidgets as widgets
from traitlets import Unicode, validate, Bool, Any, List, observe
from ipywidgets import Color
from skimage import io
from skimage.transform import resize


class ImageLabelerWidget(widgets.DOMWidget):
    _view_name = Unicode('ImageLabelerView').tag(sync=True)
    _view_module = Unicode('ImageLabeler').tag(sync=True)
    # Atributos sincronos entre js y python
    image = Unicode('blank.png').tag(sync=True)
    rect = List().tag(sync=True)
    
    Phytolith_type = Any().tag(sync=True)
    
    def __init__(self, *pargs, **kwargs):
        super(widgets.DOMWidget, self).__init__(*pargs, **kwargs)
        self._rectList = []
        # Para patron observer
        self._rectList_observers = []
    
    '''Método que se encarga de recibir la notificación
    de la adicción de unas nuevas coordenadas a la imagen'''
    @observe('rect')
    def new_rect(self, change):
        rec = change["new"]
        self._rectList.append(rec)
        # Callback a observers
        for callback in self._rectList_observers:
            callback(self._rectList)
    
    '''Método para enlazar al observer con
    el atributo rectList'''
    def rectList_bind_to(self, callback):
        self._rectList_observers.append(callback)
        
    def get_rectList(self):
        return self._rectList
    
    def set_rectList(self, value):
        self._rectList = value

In [46]:
''' Rescalamos las coordenadas para que coincidan 
las coordenadas de la vista con las de la imagen original'''
def transform_coords(coords, svg_size, image_size):
    
    real_image_h, real_image_w, _ = image_size
    # TODO Cuidado con si es menor de  svg_size x svg_size

    # Rescalado de la imagen en la vista
    if real_image_h > real_image_w:
        coef = real_image_h / svg_size
        svg_image_h = svg_size
        svg_image_w = round(real_image_w / coef)
    else:
        coef = real_image_w / svg_size
        svg_image_w = svg_size
        svg_image_h = round(real_image_h / coef)
        
    # Coeficientes de rescalado
    coef_h = real_image_h / svg_image_h
    coef_w = real_image_w / svg_image_w
    
    # Calculamos padding en función de
    # los valores de ancho y alto de la imagen
    # en la vista
    svg_padding_h = (svg_size - svg_image_h) / 2
    svg_padding_w = (svg_size - svg_image_w) / 2
    
    #TODO comprobar que las coordenadas no se salen de la imagen
    
    new_coords = []
    # Por cada rectangulo generamos una nueva imagen
    for rect_y, rect_x, rect_width, rect_height in coords:
        rect_x = round((rect_x - svg_padding_h) * coef_h)
        rect_y = round((rect_y - svg_padding_w) * coef_w)
        rect_height = round(rect_height * coef_h)
        rect_width = round(rect_width * coef_w)
        new_coords.append((rect_x, rect_x+rect_height, rect_y, rect_y+rect_width))   
    return new_coords


def save_coords_as_images(coords, image, dest_path):    
    # Por cada conjunto de rectangulos generamos una nueva imagen
    for rect_x, rect_x2, rect_y, rect_y2 in coords:
        name = str(rect_x)+str(rect_y)+".jpg"
        img = image[rect_x:rect_x2,rect_y:rect_y2]
        io.imsave(dest_path+name, img)

In [47]:
%%javascript
require.undef('ImageLabeler');

var startX, startY;
var count = 0;
var rect = null;
var rectsList = [];
var model;
var myThis;

function getMousePos(evt) {
    // Obtenemos posición absoluta del elemento svg
    var svgRef = document.getElementById("svg");
    var htmlPos = svgRef.getBoundingClientRect();
    return {
        x: evt.clientX - htmlPos.left,
        y: evt.clientY - htmlPos.top
    };
}

function updateRect(eve, rect){
    var pos = getMousePos(eve);
    var endX = pos.x;
    var endY = pos.y;
    if (endX - startX > 0){
        var x = startX
    }else{
        var x = endX;
    }

    if (endY - startY > 0){
        var y = startY
    }else{
        var y = endY;
    }
    var width = Math.abs(endX - startX);
    var height = Math.abs(endY - startY);
    
    rect.setAttributeNS(null, 'x', x);
    rect.setAttributeNS(null, 'y', y);
    rect.setAttributeNS(null, 'height', height);
    rect.setAttributeNS(null, 'width', width);
}

function setStyleRect(rect){
    rect.setAttributeNS(null, 'fill', 'transparent');
    rect.setAttributeNS(null, 'stroke', 'green');
    rect.setAttributeNS(null, 'linewidth', '10px');
}

function mouseMove(eve) {
    if(rect){
        updateRect(eve, rect);
      }
}

function mouseClick(eve) {
    if (rect !== null) {
        //Cada vez que finalizamos el rectangulo
        // , le añadimos a la lista
        console.log(rect.x['baseVal'].value,rect.y['baseVal'].value,rect.width['baseVal'].value,rect.height['baseVal'].value);
        rectsList = [];
        model.set('rect',[rect.x['baseVal'].value,rect.y['baseVal'].value,rect.width['baseVal'].value,rect.height['baseVal'].value]);
        myThis.touch();
        rect = null;
        console.log("finsihed.");
    } else {
        var pos = getMousePos(eve);
        
        startX= pos.x;
        startY = pos.y;
        
        console.log("begun.");
        
        if(!rect){
            rect = document.createElementNS("http://www.w3.org/2000/svg", 'rect');
            rect.id = 'rect' + count;
            count = count + 1;
            
            updateRect(eve, rect);

            setStyleRect(rect);
            document.getElementById('svg').appendChild(rect);
          }
        }
}

// Funciones para la carga y cambio de imagen
function loadImage(svg, src){
    svg.innerHTML = '<image id="image" xlink:href="'+ src +'" x="0" y="0" height="400px" width="400px"/>'
}

function changeImage(svg, el, image){
    console.log("Cambio de imagen");
    console.log(image);
    svg.innerHTML = '<image id="image" xlink:href="' + image + '" x="0" y="0" height="400px" width="400px"/>';
    el = svg
}


// Definición del Widget en javasript
define('ImageLabeler', ["jupyter-js-widgets"], function(widgets) {
    var svg;
    var ImageLabelerView = widgets.DOMWidgetView.extend({
        // Renderizar vista
        render: function() {
            model = this.model;
            myThis = this;
            // Creamos el SVG
            svg = document.createElementNS("http://www.w3.org/2000/svg", "svg");
            svg.id = 'svg';
            svg.setAttribute('width', '400px');
            svg.setAttribute('height', '400px');
            
            // Asignamos listeners al SVG
            svg.addEventListener("click", mouseClick, false);
            svg.addEventListener("mousemove", mouseMove, false);
            
            // Imágen
            var src = "../rsc/img/family.jpg";
            loadImage(svg, src);
            // Listener para cambios de imagen
            this.image_changed();
            this.model.on('change:image', this.image_changed, this);
            
            // Asignamos a la vista el elemento SVG
            this.el = svg
        },
        // Función que se encarga del cambio de imagen
         image_changed: function() {
             var image = this.model.get('image');
             changeImage(svg, this.el, image);
        },
    });

    return {
        ImageLabelerView: ImageLabelerView
    };
});

<IPython.core.display.Javascript object>

In [48]:
# Inicializamos Widget etiquetador de imágenes
image_labeler = ImageLabelerWidget()

# Creamos directorios

In [49]:
# Clases para la gestión de directorios
import Directory
import Directory_Manager
# Creamos directorios
dir_list = ['Fitolito tipo 1','Fitolito tipo 2',
            'Fitolito tipo 3','Fitolito tipo 4',
            'Fitolito tipo 5','Fitolito tipo 6',
            'Fitolito tipo 7', 'Fitolito tipo 8']

dir_manager = Directory_Manager.Directory_Manager(dir_list, current_dir= dir_list[0])

In [50]:
# Añadimos un diccionario que nos permita 
# añadir las coordenadas segun el tipo de fitolito
# para guardar así la imagenes en el 
#directorio que corresponda
coords_dict = dict()

''' Función que indexa las coordenadas al tipo de
fitolito que estamos eligiendo, de manera que añadimos
al diccionario cada una de las coordenadas que dibujamos
correspondiendo a la carpeta en la que sera guardada'''
def update_coords_list(coords_list):
    
    # Coordenadas a añadir
    coords_to_add = coords_list[len(coords_list)-1]
    
    # Leemos el directorio seleccionado actualmente
    current_dir_path = dir_manager.get_current_dir_path()
    current_dir = dir_manager.get_current_dir()
    
    # Si existia ya, añadimos coords a la lista
    if current_dir_path in coords_dict:
        coords_dict[current_dir_path].append(coords_to_add)
    # Sino creamos lista y añadimos coords
    else:
        coords_dict[current_dir_path] = []
        coords_dict[current_dir_path].append(coords_to_add)

# Patrón observe sobre la lista de coordenadas
image_labeler.rectList_bind_to(update_coords_list)

# Añadimos más widgets

En esta celda añadimos el *file upload*

In [51]:
from IPython.display import display
import fileupload
import PIL.Image
import io as io2
from skimage.color import rgb2gray

''' Función que se encarga de aplicar las operaciones 
necesarias para convertir los datos obtenidos del FileUpload
en una imagen'''
def image_converter(image):
        image = io2.BytesIO(image)
        image = PIL.Image.open(image)
        image = np.array(image)
        # Guardamos imagen
        rand = np.random.randint(0,1e6)
        #os.remove(rand + ".jpg")
        io.imsave(dir_manager.get_default_dir() + str(rand) + ".jpg",image)
        return str(rand) + ".jpg"

### Inicializamos el resto de Widgets

In [52]:
from ipywidgets import HBox, VBox, Label, Layout
import re
#Inicializamos Widget de File Upload
upload_widget = fileupload.FileUploadWidget()
upload_widget.description = '(50% width, 80px height) button'

# Creamos el patron para el nombre 
# de una imagen
pattern = re.compile("^.*\.(jpeg|jpg|tif|tiff|png)$", re.IGNORECASE)

# TODO Cuando se cambia de imagen
# hay que asegurarse de que las
# distintas variables se inicializan
# a los valores por defecto

# Callback para el cambio de imagen
def _cb(change):
    change = change['owner']
    # Control de que el fichero es una imagen
    # y no otro tipo de fichero
    if pattern.match(change.filename):
        # Guardamos imagen
        image_path = dir_manager.get_default_dir() + image_converter(change.data)
        # Sincronizamos cambio
        image_labeler.image = image_path

upload_widget.observe(_cb, names='data')

In [53]:
# Selector de fitolito
btns_selector =widgets.ToggleButtons(
    options=['Fitolito tipo 1', 
             'Fitolito tipo 2', 
             'Fitolito tipo 3',
             'Fitolito tipo 4',
             'Fitolito tipo 5',
             'Fitolito tipo 6',
             'Fitolito tipo 7',
             'Fitolito tipo 8'],
    #description='Tipo de fitolito',
    disabled=False,
    # 'success', 'info', 'warning', 'danger' or ''
    button_style='',
    tooltip='Description',
    #icon='check'
)

'''Función que se encarga de hacer los cambios 
necesarios al cambiar el fitolito que se etiqueta'''
def on_phytolith_change(change):
    new_dir = change['new']
    dir_manager.change_dir(new_dir)    

# listener del cambio de fitolito
btns_selector.observe(on_phytolith_change,names='value')

In [54]:
# Añadimos el botón que se encarga de guardar
#las imágenes en el directorio correspondiente

save_btn = widgets.Button(
    description='Guardar imágenes',
    disabled=False,
    button_style='',
    tooltip='Guardar imágenes',
    icon='check'
)

'''Función que se encarga de llamar a la función 
de guardar las etiquetas como imágenes'''
def on_save_btn_click(ch):
    #Obtenemos el nombre de la imagen
    image_path = image_labeler.image
    # Cargamos imágen
    image = io.imread(image_path)

    for path, coords_set in coords_dict.items():
        # Transformamos las coordenadas
        coords_set = transform_coords(coords_set, 400, image.shape)
        # Guardamos la imagenes en sus correspondientes directorios
        save_coords_as_images(coords_set, image, path)
        
    # Eliminamos los rectangulos que añadimos para
    # evitar redundacia de coordenadas
    image_labeler.set_rectList([])

save_btn.on_click(on_save_btn_click)

### Dando formato al Widget

In [55]:
# Formato de los widgets
right_size_widget = VBox([upload_widget, btns_selector, save_btn])
right_size_widget.width = '10%'
right_size_widget.margin = '5% 5% 5% 5%'
btns_selector.padding = '10%'
upload_widget.padding = '10%'
image_labeler.margin = '5% 10% 0% 0%'
upload_widget.margin = '0 0 7% 12%'
upload_widget.width = 'auto'

# Widget

In [56]:
HBox([image_labeler,right_size_widget])